# Tutorial 1 — Basic RAG (Dense Retrieval Baseline)

This notebook implements a complete baseline RAG pipeline and makes embeddings + retrieval transparent for first-time learners.

```mermaid
flowchart LR
    A[Documents] --> B[Fixed Chunking]
    B --> C[OpenAI Embeddings]
    C --> D[Chroma Vector Index]
    E[User Query] --> F[Query Embedding]
    F --> D
    D --> G[Top-k Chunks]
    G --> H[LLM Answer]
```

Continuity note:
- Tutorial 2 keeps the same pipeline but changes **chunking**.
- Tutorial 3 keeps chunking and adds **reranking**.
- Tutorial 4 adds **hybrid retrieval** (keyword + dense).

## Learning checkpoint: what works vs what breaks

**What works in Tutorial 1**
- Dense retrieval can find generally related handbook content.
- End-to-end RAG flow is functional (ingest → chunk → embed → retrieve → answer).

**Challenges you should observe**
- Query intent can be too broad for nearest-neighbor retrieval.
- Exception-heavy policy questions may return partially relevant chunks.
- Exact policy identifiers (like forms/codes) are often weakly handled.

**Why move to Tutorial 2**
- The first bottleneck is chunk quality.
- We next improve *how text is split* so policy context stays intact before retrieval.

In [ ]:
# 1) Set Up Environment and Dependencies

import importlib
import os
import shutil
import subprocess
import sys
from pathlib import Path

# Ensure uv is available (installs with: pip install uv)
if shutil.which("uv") is None:
    print("uv not found. Installing with pip...")
    subprocess.run([sys.executable, "-m", "pip", "install", "uv"], check=True)

# Ensure notebook runs from repo root and local src/ is importable
cwd = Path.cwd().resolve()
repo_root = next(
    (path for path in [cwd, *cwd.parents] if (path / "pyproject.toml").exists() and (path / "src").exists()),
    cwd,
)
os.chdir(repo_root)
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

REQUIRED_PACKAGES = [
    "openai",
    "chromadb",
    "numpy",
    "pandas",
    "rank_bm25",
    "sentence_transformers",
    "dotenv",
]

PIP_NAME_MAP = {
    "rank_bm25": "rank-bm25",
    "sentence_transformers": "sentence-transformers",
    "dotenv": "python-dotenv",
}


def find_missing(packages: list[str]) -> list[str]:
    """Return package import names not available in current kernel."""
    importlib.invalidate_caches()
    return [pkg for pkg in packages if importlib.util.find_spec(pkg) is None]


missing = find_missing(REQUIRED_PACKAGES)
if missing:
    print("Missing packages:", missing)
    print("Running: uv sync")
    subprocess.run(["uv", "sync"], check=True)

missing_after_sync = find_missing(REQUIRED_PACKAGES)
if missing_after_sync:
    print("Still missing in active kernel after uv sync:", missing_after_sync)
    pip_targets = [PIP_NAME_MAP.get(pkg, pkg) for pkg in missing_after_sync]
    print("Installing into current kernel with pip:", pip_targets)
    subprocess.run([sys.executable, "-m", "pip", "install", *pip_targets], check=True)

final_missing = find_missing(REQUIRED_PACKAGES)
if final_missing:
    raise ImportError(f"Dependencies still missing in current kernel: {final_missing}")

print("All required packages are available.")
print("Python:", sys.version.split()[0])
print("Working directory:", Path.cwd())
print("Repo root:", repo_root)
print("Using src path:", src_path)

Missing packages: ['rank_bm25']
Running: uv sync
Python: 3.11.13
Working directory: /Users/avy/GitHubProjects/allagents/all-things-rag
Repo root: /Users/avy/GitHubProjects/allagents/all-things-rag
Using src path: /Users/avy/GitHubProjects/allagents/all-things-rag/src


Resolved 204 packages in 4ms
Audited 180 packages in 43ms


In [13]:
# 2) Define Configuration and Paths

from dataclasses import dataclass
from dotenv import load_dotenv

load_dotenv()

@dataclass
class Config:
    embedding_model: str = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
    chat_model: str = os.getenv("OPENAI_CHAT_MODEL", "gpt-4.1-mini")
    chunk_mode: str = "fixed"
    top_k: int = 5
    sample_eval_size: int = 20
    handbook_path: str = "data/handbook_manual.txt"
    queries_path: str = "data/queries.jsonl"

cfg = Config()

if not os.getenv("OPENAI_API_KEY"):
    raise EnvironmentError("OPENAI_API_KEY is not set. Copy .env.example to .env and set your key.")

cfg

Config(embedding_model='text-embedding-3-small', chat_model='gpt-4.1-mini', chunk_mode='fixed', top_k=5, sample_eval_size=20, handbook_path='data/handbook_manual.txt', queries_path='data/queries.jsonl')

In [14]:
# 3) Load and Normalize Source Documents (shared handbook text + query set)

from rag_tutorials.io_utils import load_handbook_documents, load_queries

if not Path(cfg.handbook_path).exists() or not Path(cfg.queries_path).exists():
    raise FileNotFoundError(
        "Shared data files are missing. Run: uv run python scripts/generate_data.py"
    )

documents = load_handbook_documents(cfg.handbook_path)
queries = load_queries(cfg.queries_path)

print("Source text:", cfg.handbook_path)
print("Parsed handbook sections:", len(documents))
print("Queries:", len(queries))
print("Sample parsed document:", documents[0])

Source text: data/handbook_manual.txt
Parsed handbook sections: 5
Queries: 200
Sample parsed document: Document(doc_id='DOC-HB-REMOTEWORK', title='Z-Tech Handbook - Remote Work', section='Remote Work', text='Z-Tech encourages remote work from home, co-working spaces, or temporary domestic locations. Employees must stay reachable during assigned timezone hours and use approved managed devices. Public Wi-Fi usage is allowed only with corporate VPN enabled.')


In [16]:
# 4) Split Documents into Chunks (fixed chunking baseline)

from dataclasses import asdict
from rag_tutorials.chunking import fixed_chunk_documents
import pandas as pd

chunks = fixed_chunk_documents(documents, chunk_size=260)

chunk_df = pd.DataFrame([asdict(c) for c in chunks])
stats = {
    "chunk_count": len(chunk_df),
    "avg_chunk_chars": chunk_df.text.map(len).mean(),
    "max_chunk_chars": chunk_df.text.map(len).max(),
}
print(stats)
chunk_df.head(20)

{'chunk_count': 6, 'avg_chunk_chars': np.float64(191.0), 'max_chunk_chars': np.int64(260)}


,chunk_id,doc_id,section,text
0,DOC-HB-REMOTEWORK-FIX-00,DOC-HB-REMOTEWORK,Remote Work,"Z-Tech encourages remote work from home, co-wo..."
1,DOC-HB-INTERNATIONALWORK-FIX-00,DOC-HB-INTERNATIONALWORK,International Work,Working from another country is capped at 14 d...
2,DOC-HB-INTERNATIONALWORK-FIX-01,DOC-HB-INTERNATIONALWORK,International Work,xposure.
3,DOC-HB-INTERNATIONALTAX-FIX-00,DOC-HB-INTERNATIONALTAX,International Tax,Employees traveling internationally may need F...
4,DOC-HB-TRAVELAPPROVAL-FIX-00,DOC-HB-TRAVELAPPROVAL,Travel Approval,International travel requests must be submitte...
5,DOC-HB-SECURITY-FIX-00,DOC-HB-SECURITY,Security,Employees handling customer data while traveli...


In [17]:
# Chunk boundary visualization (same source text, different split strategies)

from rag_tutorials.chunking import semantic_chunk_documents

section_doc = next(doc for doc in documents if doc.section == "International Work")
fixed_view = [c.text for c in fixed_chunk_documents([section_doc], chunk_size=120)]
semantic_view = [c.text for c in semantic_chunk_documents([section_doc])]

print("Section:", section_doc.section)
print("\nFixed chunks:")
for idx, chunk_text in enumerate(fixed_view, start=1):
    print(f"[{idx}] {chunk_text}")

print("\nSemantic chunks:")
for idx, chunk_text in enumerate(semantic_view, start=1):
    print(f"[{idx}] {chunk_text}")

Section: International Work

Fixed chunks:
[1] Working from another country is capped at 14 days in a rolling 12-month period without permit support. Beyond 14 days, e
[2] mployees must open a Global Mobility case and obtain HR, Legal, and Payroll approval. Violations can trigger immigration
[3] , payroll, and tax exposure.

Semantic chunks:
[1] Working from another country is capped at 14 days in a rolling 12-month period without permit support. Beyond 14 days, employees must open a Global Mobility case and obtain HR, Legal, and Payroll approval
[2] Violations can trigger immigration, payroll, and tax exposure.


## Novice Lens: How Embeddings and Retrieval Actually Work

```mermaid
sequenceDiagram
    participant U as User Query
    participant E as Embedding Model
    participant V as Vector Store
    participant L as LLM
    U->>E: "working from another country"
    E->>V: query vector
    V-->>U: top-k chunks + scores
    U->>L: question + retrieved chunks
    L-->>U: grounded answer
```

We will inspect:
1. Query and chunk vectors (dimensions and first values)
2. Cosine similarities
3. Ranked chunk IDs returned to the generator

In [19]:
# 5) Create Embeddings and Build Vector Index

from rag_tutorials.pipeline import build_dense_retriever
from rag_tutorials.embeddings import embed_texts, cosine_similarity
import numpy as np

dense_retriever, doc_vectors = build_dense_retriever(
    chunks=chunks,
    collection_name="tutorial1_basic_dense",
    embedding_model=cfg.embedding_model,
)

print("Embedding matrix shape:", doc_vectors.shape)
print("Example vector (first 10 dims):", np.round(doc_vectors[0][:10], 4))

# Tiny toy example for intuition
sample_chunks = [chunks[i].text for i in range(3)]
sample_vectors = embed_texts(sample_chunks, model=cfg.embedding_model)
sample_query = "What is the policy for working from another country?"
sample_query_vector = embed_texts([sample_query], model=cfg.embedding_model)[0]

scores = cosine_similarity(sample_query_vector, sample_vectors)
for idx, score in enumerate(scores, start=1):
    print(f"Toy chunk {idx} cosine score: {score:.4f}")

Embedding matrix shape: (6, 1536)
Example vector (first 10 dims): [ 0.0264  0.0545  0.0487  0.0199  0.0011 -0.0401  0.0057  0.0545  0.0133
  0.002 ]
Toy chunk 1 cosine score: 0.3994
Toy chunk 2 cosine score: 0.5843
Toy chunk 3 cosine score: 0.1097


In [ ]:
# 6) Implement Retriever Logic

import pandas as pd

def retrieve_dense(question: str, top_k: int = 5):
    return dense_retriever(question, top_k=top_k)

probe_query = "What is the policy for working from another country?"
probe_results = retrieve_dense(probe_query, top_k=cfg.top_k)

pd.DataFrame([
    {
        "rank": idx + 1,
        "chunk_id": row.chunk_id,
        "score": row.score,
        "source": row.source,
        "preview": row.text[:120],
    }
    for idx, row in enumerate(probe_results)
])

,rank,chunk_id,score,source,preview
0,1,DOC-HB-INTERNATIONALWORK-FIX-00,0.168573,dense,Working from another country is capped at 14 d...
1,2,DOC-HB-INTERNATIONALTAX-FIX-00,-0.017738,dense,Employees traveling internationally may need F...
2,3,DOC-HB-REMOTEWORK-FIX-00,-0.201170,dense,"Z-Tech encourages remote work from home, co-wo..."
3,4,DOC-HB-TRAVELAPPROVAL-FIX-00,-0.220997,dense,International travel requests must be submitte...
4,5,DOC-HB-SECURITY-FIX-00,-0.323462,dense,Employees handling customer data while traveli...


In [24]:
# 7) Implement Prompt Template and LLM Call

from rag_tutorials.qa import answer_with_context

def rag_answer(question: str, top_k: int = 5):
    retrieved = retrieve_dense(question, top_k=top_k)
    context = [r.text for r in retrieved]
    print("Retrieved context:", context)
    answer = answer_with_context(question, context, model=cfg.chat_model)
    return answer, retrieved

print("Probe question:", probe_query)
answer, retrieved = rag_answer(probe_query)
print(answer)

Probe question: What is the policy for working from another country?
Retrieved context: ['Working from another country is capped at 14 days in a rolling 12-month period without permit support. Beyond 14 days, employees must open a Global Mobility case and obtain HR, Legal, and Payroll approval. Violations can trigger immigration, payroll, and tax e', 'Employees traveling internationally may need Form A-12 before departure when cross-border work exceeds 7 business days. The tax team uses Form A-12 to assess treaty relief, withholding obligations, and permanent establishment risk.', 'Z-Tech encourages remote work from home, co-working spaces, or temporary domestic locations. Employees must stay reachable during assigned timezone hours and use approved managed devices. Public Wi-Fi usage is allowed only with corporate VPN enabled.', 'International travel requests must be submitted at least 14 days before departure in the travel portal. Manager approval is mandatory, and finance approval i

In [22]:
# 8) Assemble End-to-End RAG Pipeline + 9/10 Smoke Tests and Evaluation

from rag_tutorials.evaluation import evaluate_single, summarize

sample_queries = queries[: cfg.sample_eval_size]
rows = [
    evaluate_single(
        query=q,
        retrieval_fn=lambda question: retrieve_dense(question, top_k=cfg.top_k),
        answer_fn=lambda question, context: answer_with_context(question, context, model=cfg.chat_model),
        top_k=cfg.top_k,
    )
    for q in sample_queries
]

metrics = summarize(rows)
print("Tutorial 1 metrics:", metrics)

# Show one trace row for novice debugging
trace = sample_queries[0]
trace_answer, trace_retrieved = rag_answer(trace.question, top_k=cfg.top_k)
print("\nQuery:", trace.question)
for idx, row in enumerate(trace_retrieved, start=1):
    print(f"{idx}. {row.chunk_id} | score={row.score:.4f} | {row.text[:100]}")
print("\nAnswer:", trace_answer)

Tutorial 1 metrics: {'recall_at_k': 1.0, 'mrr': 0.975, 'latency_ms': 1448.2180999591947, 'groundedness': 0.7909833712358065}

Query: Can I work remotely from cafes and home?
1. DOC-HB-REMOTEWORK-FIX-00 | score=0.1268 | Z-Tech encourages remote work from home, co-working spaces, or temporary domestic locations. Employe
2. DOC-HB-INTERNATIONALWORK-FIX-00 | score=-0.2463 | Working from another country is capped at 14 days in a rolling 12-month period without permit suppor
3. DOC-HB-INTERNATIONALTAX-FIX-00 | score=-0.3520 | Employees traveling internationally may need Form A-12 before departure when cross-border work excee
4. DOC-HB-SECURITY-FIX-00 | score=-0.3534 | Employees handling customer data while traveling must use VPN, hardware-backed MFA, and encrypted st
5. DOC-HB-TRAVELAPPROVAL-FIX-00 | score=-0.5321 | International travel requests must be submitted at least 14 days before departure in the travel port

Answer: Yes, you can work remotely from home or cafes (as co-working spaces 